# Casing design affects frac geometry and economics
_Ohm Devani_

NOTE: This project will focus on relative economic uplift between 2 casing designs in similar geology.

## Contents
1. Model inputs
2. Base case
    - define per-frac endpoints (min rate defined by dfit initiation rate, max rate defined by hhp of fleet, water volume/frac)
    - economic constants (pump charge, water cost
3. Economic sensitivities
    1. Frac surface area (PKN) vs rate; 2 series for different casing sizes
        - \$ cost/surface area vs rate vs (y-axis 2) surface area
        - Talk about impact of other variables: stress shadow, additional height growth in absence of frac barriers, better proppant/fluid distribution between clusters
    2. Frac pump time vs rate vs $ cost/stage (y2)


### Model parameters

0.8 psi/ft frac gradient
1.05 connate water SG

Horizontal well
Midpoint lateral TVD 10,000 ft

Scenario A:
7" 32# T-95 intermediate casing to 9000 ft (6.094" ID)
4 1/2" 11.6# P-110 liner to 20000 ft (4" ID)

Scenario B:
5.5" 17# P-110 casing to 20000 ft (4.892" ID)





10 clusters/stage
15 ft cluster spacing
60 bbl/ft
ignore proppant
7000 psi target surface pressure
2000 psi target NWB friction

Intermediate 

### Pipe friction estimates

Hazen-Williams for pipe friction
SPE 146674 correlation for friction reduction

In [2]:
import pandas as pd

def hwfriclosspsi(cfactor,ratebpm,pipeidinch,lengthft):
    hyddiam = 4*(3.14159*(pipeidinch/2)**2)/(2*3.14169*pipeidinch/2)
    hwfriclosspsi = 0.2083*((100/cfactor)**1.852)*((ratebpm*42)**1.852)/(hyddiam**4.8655)*.433*lengthft/100
    return hwfriclosspsi

def freduction(pipeidinch,ratebpm):
    velocftpersec = (5.615*144/60)*ratebpm/(3.14159*(pipeidinch/2)**2)
    fr = 6.2126*velocftpersec**0.5858
    return fr

#scenario_a_pipe_friction = hwfriclosspsi(120,80,)

bpmrange=list(range(0, 150))
pipefric=pd.DataFrame(bpmrange, columns = ['Surface Pumping Rate'])

print(hwfriclosspsi(120,80,4.892,15000))
print(freduction(4.892,80))

AttributeError: module 'pandas' has no attribute 'compat'